In [1]:
#hide
#skip
%config Completer.use_jedi = False
# upgrade fastrl on colab
! [ -e /content ] && pip install -Uqq fastrl['dev'] pyvirtualdisplay && \
                     apt-get install -y xvfb python-opengl > /dev/null 2>&1 
# NOTE: IF YOU SEE VERSION ERRORS, IT IS SAFE TO IGNORE THEM. COLAB IS BEHIND IN SOME OF THE PACKAGE VERSIONS

In [2]:
# hide
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbverbose.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON
else:
    # Virutual display is needed for colab
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(400, 300))
    display.start()

In [3]:
# default_exp agents.dqn.core

In [4]:
# export
# Python native modules
import os
from collections import deque
# Third party libs
import torch
from torch.nn import *
from fastcore.all import *
from fastai.learner import *
from fastai.torch_basics import *
from fastai.torch_core import *
from fastai.callback.all import *
# Local modules
from fastrl.data.block import *
from fastrl.data.gym import *
from fastrl.agent import *
from fastrl.core import *
from fastrl.memory.experience_replay import *

# DQN Core
> Contains the minimum DQN API.

In [5]:
# export
class DQN(Module):
    def __init__(self,state_sz:int,action_sz:int,hidden=512):
        self.layers=Sequential(
            Linear(state_sz,hidden),
            ReLU(),
            Linear(hidden,action_sz),
        )
    def forward(self,x): return self.layers(x)

In [6]:
dqn=DQN(4,2)
dqn(torch.randn((10,4)))

tensor([[-0.0437, -0.1542],
        [ 0.2478,  0.7249],
        [ 0.2663,  0.4346],
        [ 0.1535,  0.2444],
        [ 0.1971,  0.2716],
        [ 0.2241,  0.3392],
        [ 0.0055,  0.3505],
        [ 0.4619,  0.6979],
        [-0.1015, -0.0173],
        [ 0.3846,  0.4448]], grad_fn=<AddmmBackward>)

In [7]:
# export
class ArgMaxFeed(AgentCallback):
    def before_action(self): 
        raw_action=self.agent.model(self.experience['state'].to(default_device()))
        self.agent.raw_action_shape=raw_action.shape
        self.agent.action=torch.argmax(raw_action,dim=1).reshape(-1,1)
        
class DiscreteEpsilonRandomSelect(AgentCallback):
    
    def __init__(self,epsilon=0.5,idx=0,min_epsilon=0.2,max_epsilon=1,max_steps=5000):
        store_attr()
    
    def before_noise(self): 
        self.mask=torch.randn(size=(self.agent.action.shape[0],))<self.epsilon
        self.experience['randomly_selected']=self.mask.reshape(-1,1)
        self.experience['epsilon']=torch.full(self.agent.action.shape,self.epsilon)
        self.experience['orignal_actions']=self.agent.action.detach().clone()
        self.agent.action[self.mask]=self.agent.action[self.mask].random_(0,self.agent.raw_action_shape[1])
        self.agent.action=self.agent.action.detach().cpu().numpy()
    
        if self.agent.model.training: 
            self.idx+=1
            self.epsilon=max(self.min_epsilon,self.max_epsilon-self.idx/self.max_steps)

In [8]:
agent=Agent(dqn,cbs=[ArgMaxFeed,DiscreteEpsilonRandomSelect(max_epsilon=0.5)]).to(default_device())
agent.do_action(state=torch.randn((10,4)).to(default_device()))


(array([[0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1]]),
 {'state': tensor([[-2.4970, -1.3726,  0.6369,  0.0151],
          [ 0.0750,  0.2606,  1.8871,  0.3553],
          [-1.4058,  0.0698,  0.7881, -0.3081],
          [-0.4657, -0.8099,  1.0114,  1.2144],
          [-0.4508, -1.2925,  0.5397, -0.1329],
          [ 0.8868,  0.1393,  0.9706,  0.1309],
          [ 0.6669, -0.3596,  0.6593, -1.3893],
          [-0.2932,  0.9846, -0.0815,  0.5941],
          [ 1.0626, -0.6207, -0.7510,  0.7719],
          [-0.6088,  0.0440,  0.6037,  0.4950]], device='cuda:0'),
  'randomly_selected': tensor([[ True],
          [ True],
          [False],
          [ True],
          [ True],
          [ True],
          [ True],
          [ True],
          [False],
          [ True]]),
  'epsilon': tensor([[0.5000],
          [0.5000],
          [0.5000],
          [0.5000],
          [0.5000],
          [0.5000],
          [0.500

In [9]:
agent.show_loop()

Start Action
   - before_preprocess: []
   - after_preprocess: []
   - before_action  : [ArgMaxFeed]
   - after_action   : []
   - before_noise   : [DiscreteEpsilonRandomSelect]
   - after_noise    : []
End Action


In [10]:
# export        
class Epsilon(Metric):
    order=30
    epsilon=0

    @property
    def value(self): return self.epsilon
    def reset(self): self.epsilon=0
    def accumulate(self,learn):
        for cb in learn.model.cbs:
            if type(cb)==DiscreteEpsilonRandomSelect:
                self.epsilon=cb.epsilon

In [23]:
# export
class DQNTrainer(Callback):
    "Performs traditional training on `next_q`. Requires a callback such as `RegularNextQ`"
    def __init__(self,discount=0.99,n_steps=1):
        store_attr()
        self._xb=None  
        self.n_batch=0
    
    def after_pred(self): 
        self.learn.yb=self.xb
        # self.learn.xb=self.xb
        self._xb=(self.xb,)
        self.learn.done_mask=self.xb['done'].reshape(-1,)
        self.learn.next_q=self.learn.model.model(self.xb['next_state']).max(dim=1).values.reshape(-1,1)
        self.learn.next_q[self.done_mask]=0 #xb[done_mask]['reward']
        self.learn.targets=self.xb['reward']+self.learn.next_q*(self.discount**self.n_steps)
        self.learn.pred=self.learn.model.model(self.xb['state'])
        
        t_q=self.pred.clone()
        t_q.scatter_(1,self.xb['action'],self.targets)
        # finalize the xb and yb
        self.learn.yb=(t_q,)
        
        if (self.n_batch-1)%500==0:
            print('The loss should be practically zero: ',self.loss)
            print(self.learn.pred-t_q)
        
        
        with torch.no_grad():
            self.learn.td_error=(self.pred-self.yb[0]).mean(dim=1).reshape(-1,1)**2
        
    def before_backward(self): 
        self.n_batch+=1
        self.learn.xb=self._xb

In [20]:
# hide
SHOW_TENSOR_BOARD=True
if not os.environ.get("IN_TEST", None) and SHOW_TENSOR_BOARD:
    run_tensorboard(samples_per_plugin='images=2000')

Selecting TensorBoard with logdir runs (started 0:01:38 ago; port 6006, pid 446).


In [24]:
# slow
dqn=DQN(4,2)
agent=Agent(dqn,cbs=[ArgMaxFeed,DiscreteEpsilonRandomSelect])
source=Source(cbs=[GymLoop('CartPole-v1',agent,steps_count=3,seed=None,
                           steps_delta=1,mode='rgb_array'),
                   FirstLast,
                   ResReduce(reduce_by=4)])

dls=SourceDataBlock().dataloaders([source],n=500,bs=1,num_workers=0)

exp_replay=ExperienceReplayCallback(bs=128,max_sz=256,warmup_sz=128,freeze_at_max=True)
er_tb=ExperienceReplayTensorboard(every_epoch=3)

learn=Learner(dls,agent,loss_func=MSELoss(),
              cbs=[exp_replay,DQNTrainer(n_steps=3),er_tb],
              metrics=[Reward,Epsilon,NEpisodes])

Could not do one pass in your dataloader, there is something wrong in it


In [25]:
slow=False
learn.fit(50 if slow else 1,lr=0.01,wd=0)

The loss should be practically zero:  tensor(4.2188, device='cuda:0', grad_fn=<CloneBackward>)
tensor([[ 0.0000, -1.0906],
        [ 0.0000, -2.9765],
        [-3.2709,  0.0000],
        [-2.7540,  0.0000],
        [-3.2663,  0.0000],
        [ 0.0000, -3.0438],
        [-2.7540,  0.0000],
        [ 0.0000, -3.0810],
        [-3.5919,  0.0000],
        [-4.5049,  0.0000],
        [ 0.0000, -2.8932],
        [-3.9839,  0.0000],
        [ 0.0000, -3.0096],
        [ 0.0000, -2.9741],
        [-4.9089,  0.0000],
        [ 0.0000, -3.0602],
        [-3.1133,  0.0000],
        [-4.0299,  0.0000],
        [-4.0299,  0.0000],
        [-3.9890,  0.0000],
        [ 0.0000, -0.0855],
        [ 0.0000, -2.8277],
        [ 0.0000, -2.8539],
        [ 0.0000, -3.0602],
        [-3.2663,  0.0000],
        [ 0.0000, -1.0400],
        [ 0.0000, -3.0679],
        [-4.2468,  0.0000],
        [-5.2569,  0.0000],
        [-3.2335,  0.0000],
        [ 0.0000, -2.9117],
        [-4.5285,  0.0000],
        [

In [ ]:
# hide
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.export import *
    from nbdev.export2html import *
    from nbverbose.cli import *
    make_readme()
    notebook2script()
    notebook2html()